# Trabalho Final de Aprendizado de Máquina
### Aluna: Aicha Khalid Hassan Al-Rob

##### Este trabalho utilizará como base um dataset de cinco milhões de letras de músicas, trazidos de um site de letras de músicas. O objetivo é criar um modelo de aprendizado de máquina que consiga prever o gênero musical de uma letra de música, baseado em seu conteúdo.

Vamos, primeiro, importar as bibliotecas necessárias para o trabalho.


In [ ]:
!pip install kaggle

In [5]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import requests
import kaggle
import pickle
from random import seed, sample
import glob
import os
from kaggle.api.kaggle_api_extended import KaggleApi

In [13]:
my_data = pd.read_csv('spotify_millsongdata.csv', chunksize=100000)
my_data = pd.concat(my_data, ignore_index=True)

In [16]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


Agora, vamos ao pré processamento dos dados. Primeiro, vamos ler o dataset e ver como ele está estruturado.

Como este dataset é muito grande, utilizar o método do pandas 'read_csv()' não se aplica aqui. 

In [ ]:
class Loader():
    def __init__(self,input_path,output_path):
        self.input_path = input_path
        self.output_path = output_path
        self.chunksize = 10000

    def produce_pickles(self):
        with pd.read_csv(self.input_path, chunksize= self.chunksize) as reader:
            try:
                os.makedirs(self.output_path)
            except FileExistsError:
                pass # diretorio ja existe
            for i, chunk in enumerate(reader):
                out_file = self.output_path + f'/chunk{i}.pkl'
                with open(out_file, "wb") as file:
                    pickle.dump(chunk,file, pickle.HIGHEST_PROTOCOL)

        # with pd.read_csv(self.input_path, chunksize=self.chunksize) as reader:
        #     for i, chunk in enumerate(reader):
        #         chunk.to_pickle(self.output_path + f'chunk{i}.pkl')

    def load_pickle(self, pickle_id): # carrega um arquivo pickle de acordo com o id fornecido
        # se diretorio estiver vazio ou nao existir
        if (not os.path.exists(self.output_path)) or (len(os.listdir(self.output_path)) == 0):
            self.produce_pickles()
        
        file_path = self.output_path + "/chunk" + str(pickle_id) + ".pkl"
        if os.path.isfile(file_path):
            my_df = pd.read_pickle(file_path)
        else:
            raise Exception(f"O arquivo chunk{pickle_id}.pkl não existe")
        return my_df         
    
    def random_pickles(self, n_pickles=3, init=42, verbose=True):
        # faz a leitura aleatoria de pickles. 
        # n_pickles sao a quantidade a ser carregada, init é um parametro pra randomização da leitura
        # verbose é uma booleana, se for true, mostra no terminal os arquivos carregados.
 
        if (not os.path.exists(self.output_path)) or (len(os.listdir(self.output_path)) == 0):
            self.produce_pickles()
        
        pickle_files = []
        for name in glob.glob(self.output_path + "chunk*.pkl"):
            pickle_files.append(name)
            print(len(pickle_files))
        
        seed(init)

        if len(pickle_files) >= n_pickles:
            random_p_files = sample(pickle_files, n_pickles)
        else:
            raise Exception("A quantidade de pickles fornecida excede o numero de arquivos pickle")

        df_list = []
        for pickle in random_p_files:
            pd.read_pickle(pickle) 

        my_df = pd.concat(df_list, ignore_index=True)

        if verbose:
            print("Pickles carregados na memória:")
            for pickle in random_p_files:
                print(pickle)
        
        return my_df
    
    def find_pickles(self, num,):
        # verifica se arquivo existe
        if (not os.path.exists(self.output_path)) or (len(os.listdir(self.output_path)) == 0):
            self.produce_pickles()
        

In [ ]:
my_input = './song_lyrics.csv'
my_ouput = './output/data'

my_loader = Loader(input_path=my_input, output_path=my_ouput)

my_data = my_loader.random_pickles(n_pickles=2)

In [ ]:
my_data.head()

In [ ]:
my_data.info()

In [ ]:

my_data.dtypes

In [ ]:

print(f"O dataframe está com {my_data.shape[0]} linhas e {my_data.shape[1]} colunas")
print(f"Numero de itens (musicas) únicos: {len(my_data.id.unique())}")
